# Kratos example: Mohr-Coulomb

In [2]:
from probabilistic_library import ReliabilityProject, ReliabilityMethod, DistributionType, CombineProject, CombineType, CombinerMethod, CompareType
from probabilistic_library import UncertaintyProject, UncertaintyMethod, Stochast, RunProject, RunValuesType

### Model

In [ ]:
from run_kratos_quay_wall import zfuncwrapper_mohr_coulomb

class Wrapper:
    def __init__(self):
        self.mc_wrapper = zfuncwrapper_mohr_coulomb(output_stage_number = 2, clean_up = True)

    def _expand(self, clay_cohesion, sand_cohesion, clay_friction_angle, sand_friction_angle, clay_young_modulus, sand_young_modulus, sheetpile_young_modulus, sheetpile_poisson_ratio, sheetpile_thickness):

        geo_cohesions = [0] * 7
        geo_friction_angles = [0] * 7
        geo_young_modulus = [0] * 7

        for i in [0,1,2,3]:
            geo_cohesions[i] = clay_cohesion
            geo_friction_angles[i] = clay_friction_angle
            geo_young_modulus[i] = clay_young_modulus

        for i in [4,5,6]:
            geo_cohesions[i] = sand_cohesion
            geo_friction_angles[i] = sand_friction_angle
            geo_young_modulus[i] = sand_young_modulus

        int_cohesions = [0] * 6
        int_friction_angles = [0] * 6

        for i in [0,1,2,3]:
            int_cohesions[i] = clay_cohesion
            int_friction_angles[i] = clay_friction_angle
        
        for i in [4,5]:
            int_cohesions[i] = sand_cohesion
            int_friction_angles[i] = sand_friction_angle

        sp_young_modulus = [sheetpile_young_modulus]
        sp_poisson_ratio = [sheetpile_poisson_ratio]
        sp_thickness = [sheetpile_thickness]

        return geo_cohesions, geo_friction_angles, geo_young_modulus, int_cohesions, int_friction_angles, sp_young_modulus, sp_poisson_ratio, sp_thickness

    def total_displacement(self, clay_cohesion, sand_cohesion, clay_friction_angle, sand_friction_angle, clay_young_modulus, sand_young_modulus, sheetpile_young_modulus, sheetpile_poisson_ratio, sheetpile_thickness):
        lists = self._expand(clay_cohesion, sand_cohesion, clay_friction_angle, sand_friction_angle, clay_young_modulus, sand_young_modulus, sheetpile_young_modulus, sheetpile_poisson_ratio, sheetpile_thickness)
        displacement = self.mc_wrapper.total_displacement(lists[0], lists[1], lists[2], lists[3], lists[4], lists[5], lists[6], lists[7])
        return displacement





### Variables

In [4]:
def define_project_variables(project):

    project.variables["clay_cohesion"].distribution = DistributionType.log_normal
    project.variables["clay_cohesion"].mean = 3.0
    project.variables["clay_cohesion"].variation = 0.2

    project.variables["sand_cohesion"].distribution = DistributionType.log_normal
    project.variables["sand_cohesion"].mean = 3.0
    project.variables["sand_cohesion"].variation = 0.2

    project.variables["clay_friction_angle"].distribution = DistributionType.log_normal
    project.variables["clay_friction_angle"].mean = 22.5
    project.variables["clay_friction_angle"].variation = 0.08

    project.variables["sand_friction_angle"].distribution = DistributionType.log_normal
    project.variables["sand_friction_angle"].mean = 32.5
    project.variables["sand_friction_angle"].variation = 0.08

    project.variables["clay_young_modulus"].distribution = DistributionType.uniform
    project.variables["clay_young_modulus"].minimum = 5*10**6
    project.variables["clay_young_modulus"].maximum = 8*10**6
        
    project.variables["sand_young_modulus"].distribution = DistributionType.uniform
    project.variables["sand_young_modulus"].minimum = 30*10**6
    project.variables["sand_young_modulus"].maximum = 50*10**6

    project.variables["sheetpile_young_modulus"].distribution = DistributionType.deterministic
    project.variables["sheetpile_young_modulus"].mean = 200e9 # value checked

    project.variables["sheetpile_poisson_ratio"].distribution = DistributionType.deterministic
    project.variables["sheetpile_poisson_ratio"].mean = 0.0 # value checked

    project.variables["sheetpile_thickness"].distribution = DistributionType.deterministic
    project.variables["sheetpile_thickness"].mean = 0.02159 # value checked


### Perform a single run

In [6]:
run_project = RunProject()
wrapper = Wrapper()

run_project.model = wrapper.total_displacement

define_project_variables(run_project)

run_project.model.print()

Model total_displacement:
Input parameters:
  clay_cohesion
  sand_cohesion
  clay_friction_angle
  sand_friction_angle
  clay_young_modulus
  sand_young_modulus
  sheetpile_young_modulus
  sheetpile_poisson_ratio
  sheetpile_thickness
Output parameters:
  displacement


In [ ]:

run_project.settings.run_values_type = RunValuesType.mean_values
run_project.run()

realization = run_project.realization

realization.print()

### Uncertainty analysis (total displacement)

In [ ]:
project = UncertaintyProject()
wrapper = zfuncwrapper_mohr_coulomb(output_stage_number = 2, clean_up = True)
project.model = wrapper.total_displacement

In [ ]:
define_project_variables(project)

project.settings.uncertainty_method = UncertaintyMethod.crude_monte_carlo
project.settings.variation_coefficient = 0.02
project.settings.minimum_samples = 10
project.settings.maximum_samples = 50
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.stochast.plot()

### Uncertainty analysis (max bending moment)

In [ ]:
project = UncertaintyProject()
wrapper = zfuncwrapper_mohr_coulomb(output_stage_number = 3, clean_up = True)
project.model = wrapper.max_bending_moment

project.model.print()

In [ ]:
project = define_project_variables(project)

project.settings.uncertainty_method = UncertaintyMethod.crude_monte_carlo
project.settings.variation_coefficient = 0.02
project.settings.minimum_samples = 10
project.settings.maximum_samples = 50
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.stochast.plot()

### Reliability analysis (total displacement)

In [ ]:
project = ReliabilityProject()
wrapper = zfuncwrapper_mohr_coulomb(output_stage_number = 3, clean_up = True)
project.model = wrapper.total_displacement

project.model.print()

In [ ]:
project = define_project_variables(project)

project.limit_state_function.parameter = project.model.output_parameters[0]
project.limit_state_function.compare_type = CompareType.greater_than
project.limit_state_function.critical_value = 0.1

# form does not work!
project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
project.settings.variation_coefficient = 0.02
project.settings.minimum_samples = 50
project.settings.maximum_samples = 100
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.design_point.print()

dp1 = project.design_point

### Reliability analysis (max bending moment)

In [ ]:
wrapper = zfuncwrapper_mohr_coulomb(output_stage_number = 2, clean_up = True)
project.model = wrapper.max_bending_moment

project.model.print()

In [ ]:
project = define_project_variables(project)

project.limit_state_function.parameter = project.model.output_parameters[0]
project.limit_state_function.compare_type = CompareType.greater_than
project.limit_state_function.critical_value = 15400

# form does not work!
project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
project.settings.variation_coefficient = 0.02
project.settings.minimum_samples = 50
project.settings.maximum_samples = 100
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.design_point.print()

dp2 = project.design_point

### Reliability analysis - system

In [ ]:
combine_project = CombineProject()

combine_project.design_points.append(dp1)
combine_project.design_points.append(dp2)

combine_project.settings.combine_type = CombineType.series
combine_project.settings.combiner_method = CombinerMethod.hohenbichler_form

combine_project.run()

combine_project.design_point.print()